In [1]:
import numpy as np
import pandas as pd
import wfdb

In [2]:
# path to mimic_ecg_dataset (output from create_mimic_ecg_dataset.ipynb)
df = pd.read_csv("../data/mimic_ecg_dataset.csv")

In [3]:
len(df)

180686

In [4]:
df.columns

Index(['subject_id', 'study_id', 'file_name', 'ecg_time', 'path', 'edregtime',
       'ed_stay_id', 'hadm_id', 'REVASC', 'CABG', 'PCI', 'CAG', 'gender',
       'anchor_age', 'first_troponin_t', 'second_troponin_t',
       'third_troponin_t', 'troponin_t_delta', 'first_ck', 'second_ck',
       'third_ck', 'ck_delta', 'first_ckmb', 'second_ckmb', 'third_ckmb',
       'ckmb_delta', 'rhythm', 'ventricular_rate', 'pr_interval',
       'p_wave_duration', 'qrs_duration', 'qt_interval', 'qtc_interval',
       'p_axis', 'qrs_axis', 't_axis', 'sinus_rhythm', 'diabetes',
       'hypertension', 'hyperlipidemia', 'old_mi', 'stemi', 'nstemi',
       'suggestive_symptoms', 'chest_pain', 'dyspnea', 'palpitations',
       'race_white', 'race_black', 'race_hispanic', 'race_asian', 'race_other',
       'arrival_walk_in', 'arrival_ambulance', 'arrival_unknown'],
      dtype='object')

In [5]:
y_pci = df["REVASC"].values
y_coro = df["CAG"].values
groups = df["subject_id"].values
gender = df["gender"].values
age = df["anchor_age"].values

In [6]:
# path to MIMIC ECGs
pre_path = ".....mimic-iv-ecg-diagnostic-electrocardiogram-matched-subset-1.0/"

k = len(df)

X = np.zeros((k, 12, 1000))
for i in range(k):
    if i % 10000 == 0:
        print(i)
    filepath = pre_path + df["path"].iloc[i]
    data = np.asarray(wfdb.rdsamp(filepath)[0]).T.reshape((1, 12, 5000))
    # sampling rate to 100 HZ and switch leads to match up with control cohort Lano
    X[i] = data[0, [0, 1, 2, 3, 5, 4, 6, 7, 8, 9, 10, 11], ::5] 

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000


In [7]:
np.savez("./revasc_model/data/mimic_ecgs_and_labels_age_gender.npz", 
         X=X, 
         y_revasc=df["REVASC"].values, 
         y_cag=df["CAG"].values, 
         subject_id=df["subject_id"].values,
         gender = df["gender"].values,
         age = df["anchor_age"].values,
         file_name=df["file_name"].values
        )